In [ ]:
import sys
import random
import numpy as np
import os
from PIL import Image
from src.env.env import RILAB_OMY_ENV
import gc, argparse
import json
import copy
import time
from src.controllers import load_controller
from lerobot.common.datasets.lerobot_dataset import LeRobotDataset
import glfw

In [ ]:
# Load environment configuration
config_file_path = './configs/train_key.json'
with open(config_file_path) as f:
    env_conf = json.load(f)
language_instruction = env_conf['language_instruction']
omy_env = RILAB_OMY_ENV(cfg=env_conf, seed=None, 
                        action_type=env_conf['control_mode'], 
                        obs_type='full_state',
                        vis_mode = 'keyboard')


omy_env.reset(leader_pose = False)
# Load keyboard controller
controller = load_controller('keyboard',env_conf)
controller.reset(omy_env)

You can teleop your robot with keyboard
```
---------     -----------------------
   w       ->        backward
s  a  d        left   forward   right
---------      -----------------------
In x, y plane

---------
R: Moving Up
F: Moving Down
---------
In z axis

---------
Q: Tilt left
E: Tilt right
UP: Look Upward
Down: Look Donward
Right: Turn right
Left: Turn left
---------
For rotation

---------
SPACEBAR: Toggle Gripper
--------

---------
z: reset
--------
```

In [ ]:
while omy_env.env.is_viewer_alive():
    omy_env.step_env()
    if omy_env.env.loop_every(HZ=20):
        temp = omy_env.get_object_pose(pad=5)
        delta_eef = controller.get_action()
        omy_env.step(delta_eef)
        omy_env.grab_image()
        omy_env.render(task=language_instruction)
        success = omy_env.check_success()
        if omy_env.env.is_key_pressed_once(glfw.KEY_Z):
            omy_env.reset(leader_pose = False)
        if success or omy_env.env.is_key_pressed_once(glfw.KEY_ESCAPE):
            break
    omy_env.env.sync_sim_wall_time()
omy_env.env.close_viewer()
print("Episode ended.")
print(f"Success: {success}")